In [1]:
# ===== Imports & config =====
import numpy as np
import pandas as pd
from google.colab import drive
from sqlalchemy import create_engine, text
from sqlalchemy.types import Integer, Float, String, Date, DateTime, Boolean

CITIBIKE_CSV = "/content/drive/MyDrive/citibike_cleaned.csv"
WEATHER_CSV  = "/content/drive/MyDrive/weather_cleaned.csv"
DB_PATH      = "/content/citibike.db"

# ===== Load cleaned CSVs from Drive =====
drive.mount("/content/drive")

citibike = pd.read_csv(
    CITIBIKE_CSV, low_memory=False, parse_dates=["start_time", "stop_time"]
)
weather = pd.read_csv(
    WEATHER_CSV, low_memory=False, parse_dates=["date"]
)

# ===== Build relational dataframes in memory =====
# -- stations (dedupe by station_id using name mode + median lat/lon)
starts = citibike[["start_station_id","start_station_name","start_latitude","start_longitude"]].rename(
    columns={"start_station_id":"station_id","start_station_name":"name",
             "start_latitude":"latitude","start_longitude":"longitude"}
)
ends = citibike[["end_station_id","end_station_name","end_latitude","end_longitude"]].rename(
    columns={"end_station_id":"station_id","end_station_name":"name",
             "end_latitude":"latitude","end_longitude":"longitude"}
)
stations = pd.concat([starts, ends], ignore_index=True)
stations = stations.dropna(subset=["station_id"]).copy()
stations["station_id"] = pd.to_numeric(stations["station_id"], errors="coerce").astype(int)
stations["name"] = stations["name"].astype("string").str.strip().str.replace(r"\s+", " ", regex=True)
stations = (
    stations.groupby("station_id", as_index=False)
            .agg({"name": lambda s: s.value_counts().index[0],  # mode
                  "latitude": "median",
                  "longitude": "median"})
)

# -- trips (select expected cols + practical dtypes)
trip_cols = [
    "start_time","stop_time","start_station_id","end_station_id","bike_id",
    "user_type","birth_year","gender","trip_duration_seconds","trip_duration_minutes",
    "distance_km","speed_kmh","latitude_change","longitude_change","start_hour",
    "start_weekday","is_weekend","rush_hour","start_date","age"
]
trips = citibike.loc[:, [c for c in trip_cols if c in citibike.columns]].copy()

for c in ["start_station_id","end_station_id","bike_id","birth_year","start_hour","age"]:
    if c in trips: trips[c] = pd.to_numeric(trips[c], errors="coerce").astype("Int64")
for c in ["trip_duration_seconds","trip_duration_minutes","distance_km","speed_kmh",
          "latitude_change","longitude_change"]:
    if c in trips: trips[c] = pd.to_numeric(trips[c], errors="coerce")
for c in ["is_weekend","rush_hour"]:
    if c in trips: trips[c] = trips[c].astype("boolean")
for c in ["start_weekday","user_type","gender"]:
    if c in trips: trips[c] = trips[c].astype("string")
if "start_date" in trips and trips["start_date"].dtype == "object":
    trips["start_date"] = pd.to_datetime(trips["start_date"]).dt.date

# -- weather (keep exactly the modeled columns; coerce date)
weather = weather.copy()
weather["date"] = pd.to_datetime(weather["date"]).dt.date
weather_expected = [
    "date","station","name",
    "average_daily_wind_speed_mph","precipitation_in","snowfall_in","snow_depth_in",
    "average_temp_f","max_temp_f","min_temp_f",
    "wind_direction_fastest_2_min_degrees",
    "wind_speed_fastest_2_min_mph","wind_speed_fastest_5_sec_mph",
]
for c in weather_expected:
    if c not in weather.columns:
        weather[c] = np.nan
weather = weather[weather_expected]

# ===== Create SQLite schema =====
engine = create_engine(f"sqlite:///{DB_PATH}", future=True)

ddl_stations = """
CREATE TABLE IF NOT EXISTS stations (
  station_id INTEGER PRIMARY KEY,
  name       TEXT,
  latitude   REAL,
  longitude  REAL
);
"""
ddl_trips = """
CREATE TABLE IF NOT EXISTS trips (
  trip_id                INTEGER PRIMARY KEY AUTOINCREMENT,
  start_time             TIMESTAMP,
  stop_time              TIMESTAMP,
  start_station_id       INTEGER,
  end_station_id         INTEGER,
  bike_id                INTEGER,
  user_type              TEXT,
  birth_year             INTEGER,
  gender                 TEXT,
  trip_duration_seconds  REAL,
  trip_duration_minutes  REAL,
  distance_km            REAL,
  speed_kmh              REAL,
  latitude_change        REAL,
  longitude_change       REAL,
  start_hour             INTEGER,
  start_weekday          TEXT,
  is_weekend             BOOLEAN,
  rush_hour              BOOLEAN,
  start_date             DATE,
  age                    INTEGER,
  FOREIGN KEY(start_station_id) REFERENCES stations(station_id),
  FOREIGN KEY(end_station_id)   REFERENCES stations(station_id)
);
"""
ddl_weather = """
CREATE TABLE IF NOT EXISTS weather (
  date                                DATE PRIMARY KEY,
  station                             TEXT,
  name                                TEXT,
  average_daily_wind_speed_mph        REAL,
  precipitation_in                    REAL,
  snowfall_in                         REAL,
  snow_depth_in                       REAL,
  average_temp_f                      REAL,
  max_temp_f                          REAL,
  min_temp_f                          REAL,
  wind_direction_fastest_2_min_degrees REAL,
  wind_speed_fastest_2_min_mph        REAL,
  wind_speed_fastest_5_sec_mph        REAL
);
"""

with engine.begin() as conn:
    conn.execute(text("PRAGMA foreign_keys = ON;"))
    conn.execute(text(ddl_stations))
    conn.execute(text(ddl_trips))
    conn.execute(text(ddl_weather))

# ===== Truncate (safe re-runs) & load tables =====
with engine.begin() as conn:
    conn.execute(text("DELETE FROM trips;"))     # child first
    conn.execute(text("DELETE FROM weather;"))   # independent
    conn.execute(text("DELETE FROM stations;"))  # parent last

stations.to_sql(
    "stations", engine, if_exists="append", index=False, method="multi", chunksize=5000,
    dtype={"station_id": Integer(), "name": String(), "latitude": Float(), "longitude": Float()}
)
weather.to_sql(
    "weather", engine, if_exists="append", index=False, method="multi", chunksize=5000,
    dtype={
        "date": Date(), "station": String(), "name": String(),
        "average_daily_wind_speed_mph": Float(), "precipitation_in": Float(),
        "snowfall_in": Float(), "snow_depth_in": Float(),
        "average_temp_f": Float(), "max_temp_f": Float(), "min_temp_f": Float(),
        "wind_direction_fastest_2_min_degrees": Float(),
        "wind_speed_fastest_2_min_mph": Float(),
        "wind_speed_fastest_5_sec_mph": Float(),
    }
)
trips.to_sql(
    "trips", engine, if_exists="append", index=False, method="multi", chunksize=5000,
    dtype={
        "start_time": DateTime(), "stop_time": DateTime(),
        "start_station_id": Integer(), "end_station_id": Integer(),
        "bike_id": Integer(), "user_type": String(),
        "birth_year": Integer(), "gender": String(),
        "trip_duration_seconds": Float(), "trip_duration_minutes": Float(),
        "distance_km": Float(), "speed_kmh": Float(),
        "latitude_change": Float(), "longitude_change": Float(),
        "start_hour": Integer(), "start_weekday": String(),
        "is_weekend": Boolean(), "rush_hour": Boolean(),
        "start_date": Date(), "age": Integer(),
    }
)

# quick counts
with engine.begin() as conn:
    print(pd.read_sql("SELECT COUNT(*) AS n FROM stations", conn))
    print(pd.read_sql("SELECT COUNT(*) AS n FROM weather",  conn))
    print(pd.read_sql("SELECT COUNT(*) AS n FROM trips",    conn))

# ===== Creating analytics views (7 total) =====

views_sql = [

# 1) Daily ridership
"""
CREATE VIEW IF NOT EXISTS v_daily_ridership AS
SELECT
  t.start_date                          AS date,
  COUNT(*)                              AS rides,
  ROUND(AVG(t.trip_duration_minutes),2) AS avg_duration_min,
  ROUND(AVG(t.speed_kmh),2)             AS avg_speed_kmh
FROM trips t
GROUP BY t.start_date
ORDER BY t.start_date;
""",

# 2) Hourly ridership
"""
CREATE VIEW IF NOT EXISTS v_hourly_ridership AS
SELECT
  t.start_date AS date,
  t.start_hour AS hour,
  COUNT(*)     AS rides,
  ROUND(AVG(t.trip_duration_minutes),2) AS avg_duration_min
FROM trips t
GROUP BY t.start_date, t.start_hour
ORDER BY t.start_date, t.start_hour;
""",

# 3) Station-level daily starts/ends
"""
CREATE VIEW IF NOT EXISTS v_station_daily_ridership AS
WITH base AS (
  SELECT
    t.start_date,
    s.station_id,
    s.name AS station_name,
    SUM(CASE WHEN t.start_station_id = s.station_id THEN 1 ELSE 0 END) AS starts,
    SUM(CASE WHEN t.end_station_id   = s.station_id THEN 1 ELSE 0 END) AS ends
  FROM stations s
  LEFT JOIN trips t
    ON t.start_station_id = s.station_id OR t.end_station_id = s.station_id
  GROUP BY t.start_date, s.station_id, s.name
)
SELECT * FROM base
ORDER BY start_date, starts DESC, ends DESC;
""",

# 4) Daily ridership + weather
"""
CREATE VIEW IF NOT EXISTS v_daily_ridership_weather AS
SELECT
  d.date, d.rides, d.avg_duration_min, d.avg_speed_kmh,
  w.average_temp_f, w.max_temp_f, w.min_temp_f,
  w.precipitation_in, w.average_daily_wind_speed_mph,
  w.wind_speed_fastest_2_min_mph, w.wind_speed_fastest_5_sec_mph
FROM v_daily_ridership d
LEFT JOIN weather w ON w.date = d.date
ORDER BY d.date;
""",

# 5) Weather effect buckets
"""
CREATE VIEW IF NOT EXISTS v_weather_effects AS
SELECT
  CASE WHEN COALESCE(w.precipitation_in,0) > 0 THEN 'precip' ELSE 'dry' END AS precip_bucket,
  COUNT(t.rowid)                                AS rides,
  ROUND(AVG(t.speed_kmh),2)                     AS avg_speed_kmh,
  ROUND(AVG(t.trip_duration_minutes),2)         AS avg_duration_min
FROM trips t
LEFT JOIN weather w ON w.date = t.start_date
GROUP BY precip_bucket
ORDER BY precip_bucket;
""",

# 6) Top stations overall
"""
CREATE VIEW IF NOT EXISTS v_top_stations AS
WITH starts AS (
  SELECT start_station_id AS station_id, COUNT(*) AS starts
  FROM trips GROUP BY start_station_id
),
ends AS (
  SELECT end_station_id AS station_id, COUNT(*) AS ends
  FROM trips GROUP BY end_station_id
)
SELECT
  st.station_id,
  st.name AS station_name,
  COALESCE(starts.starts,0) AS starts,
  COALESCE(ends.ends,0)   AS ends,
  COALESCE(starts.starts,0) + COALESCE(ends.ends,0) AS total_activity
FROM stations st
LEFT JOIN starts ON starts.station_id = st.station_id
LEFT JOIN ends   ON ends.station_id   = st.station_id
ORDER BY total_activity DESC, st.station_id;
""",

# 7) Commute profile (weekdays)
"""
CREATE VIEW IF NOT EXISTS v_commute_profile AS
SELECT
  t.start_date AS date,
  t.start_weekday,
  t.rush_hour,
  COUNT(*) AS rides,
  ROUND(AVG(t.trip_duration_minutes),2) AS avg_duration_min
FROM trips t
WHERE t.is_weekend = 0
GROUP BY t.start_date, t.start_weekday, t.rush_hour
ORDER BY t.start_date, t.rush_hour DESC;
""",
]

with engine.begin() as conn:
    for sql in views_sql:
        conn.execute(text(sql))

print("✅ Views created.")

# ===== Preview each view (first 10 rows) =====
view_names = [
    "v_daily_ridership",
    "v_hourly_ridership",
    "v_station_daily_ridership",
    "v_daily_ridership_weather",
    "v_weather_effects",
    "v_top_stations",
    "v_commute_profile",
]

with engine.begin() as conn:
    for v in view_names:
        print(f"\n--- {v} ---")
        display(pd.read_sql(f"SELECT * FROM {v} LIMIT 10;", conn))

Mounted at /content/drive
     n
0  102
     n
0  366
        n
0  236906
✅ Views created.

--- v_daily_ridership ---


,date,rides,avg_duration_min,avg_speed_kmh
0,2016-01-01,149,8.68,8.71
1,2016-01-02,193,10.53,8.71
2,2016-01-03,261,12.63,8.44
3,2016-01-04,278,6.91,9.84
4,2016-01-05,271,6.50,9.63
5,2016-01-06,343,7.33,9.66
6,2016-01-07,429,10.17,9.88
7,2016-01-08,413,7.30,9.68
8,2016-01-09,280,9.35,9.49
9,2016-01-10,192,10.78,8.61



--- v_hourly_ridership ---


,date,hour,rides,avg_duration_min
0,2016-01-01,0,4,4.22
1,2016-01-01,1,7,7.91
2,2016-01-01,2,9,6.55
3,2016-01-01,3,8,8.66
4,2016-01-01,4,4,6.10
5,2016-01-01,5,2,14.04
6,2016-01-01,6,1,2.33
7,2016-01-01,7,2,15.02
8,2016-01-01,8,5,7.37
9,2016-01-01,9,6,8.64



--- v_station_daily_ridership ---


...

,precip_bucket,rides,avg_speed_kmh,avg_duration_min
0,dry,174002,9.14,9.83
1,precip,62904,9.19,9.85



--- v_top_stations ---


,station_id,station_name,starts,ends,total_activity
0,3186,Grove St Path,28288,37846,66134
1,3183,Exchange Place,18325,21547,39872
2,3195,Sip Ave,16910,15654,32564
3,3203,Hamilton Park,14655,14773,29428
4,3202,Newport Path,12862,13034,25896
5,3214,Essex Light Rail,8983,9482,18465
6,3209,Brunswick St,9403,8280,17683
7,3211,Newark Ave,8932,8118,17050
8,3187,Warren St,8001,7884,15885
9,3213,Van Vorst Park,8215,7293,15508



--- v_commute_profile ---


,date,start_weekday,rush_hour,rides,avg_duration_min
0,2016-01-01,Friday,1,47,8.48
1,2016-01-01,Friday,0,102,8.77
2,2016-01-04,Monday,1,189,6.61
3,2016-01-04,Monday,0,89,7.56
4,2016-01-05,Tuesday,1,184,6.78
5,2016-01-05,Tuesday,0,87,5.93
6,2016-01-06,Wednesday,1,218,6.72
7,2016-01-06,Wednesday,0,125,8.40
8,2016-01-07,Thursday,1,278,11.65
9,2016-01-07,Thursday,0,151,7.43
